In [ ]:
#For Kaggle
#date=''

#For WL data
date='20241118'
source='WL'

saveImg=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import countrymanagement as cm
import mccmanagement
reload(fd)
reload(sd)
reload(di)
reload(cm)
reload(mccmanagement)

print('done')

In [ ]:
import dataimport as di
import pandas as pd 
   
dfTrx = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Remove unless columns

In [ ]:
# remove db_uuid issuer_id cluster_profile

dfTrx= dfTrx.drop(columns=['issuer_id', 'cluster_profile'])
dfTrx.head()

# Fill missing values

In [ ]:
import numpy as np

nan_count = dfTrx.isnull().sum()
print('Number of NaN values:', nan_count)


dfTrx['mcd_fraud_score'].fillna(np.mean(dfTrx['mcd_fraud_score']), inplace=True)
dfTrx['vaa_score'].fillna(np.mean(dfTrx['vaa_score']), inplace=True)
dfTrx.head()

# Grouping countries

In [ ]:
dfTrx['country_group'] = dfTrx['term_country'].apply(cm.get_country_group)
dfTrx= dfTrx.drop(columns=['term_country'])
dfTrx.head()

# Grouping MCC

In [ ]:
dfTrx['mcc_group'] = dfTrx['term_mcc'].apply(mccmanagement.get_mcc_group_ATM)
dfTrx= dfTrx.drop(columns=['term_mcc'])
dfTrx.head()

# One hot encoding

In [ ]:
dfTrx=pd.get_dummies(dfTrx,columns=['card_brand','country_group','mcc_group'], dtype = int)
dfTrx.head()

# Remove columns not yet managed

In [ ]:
dfTrx= dfTrx.drop(columns=['TRX_3D_SECURED','trx_accepted','trx_cnp','trx_response_code','trx_reversal',
                             'ecom_indicator','trx_authentication','pos_entry_mode','card_entry_mode','ch_present',
                            'card_pan_id'])
dfTrx.head()

# Save clean file

In [ ]:
dfTrx.to_csv('../data/cleaned/'+source+'export'+date+'.csv', index=False) 

# Read saved file

In [ ]:
dfTrxSaved = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrxSaved.head(5)

In [ ]:
import dataimport as di
import pandas as pd 
   
dfTrx = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrx = di.remove_columns(dfTrx)
dfTrx = di.fill_missing_values(dfTrx)
dfTrx = di.category_grouping(dfTrx)
dfTrx = di.category_encoding(dfTrx)
dfTrx = di.remove_column_not_yet_managed(dfTrx)
dfTrx.head(5)

In [ ]:
dfTrx = di.full_import_and_clean('../data/raw/'+source+'export'+date+'.csv')
dfTrx.to_csv('../data/cleaned/'+source+'export'+date+'.csv', index=False) 
dfTrxSaved = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrxSaved.head(5)

# Import fill and clean a raw files

In [ ]:
dates =['20241118', '20241119','20241120','20241125']
for date in dates:
    print('process',date)
    dfTrx = di.full_import_and_clean('../data/raw/'+source+'export'+date+'.csv')
    dfTrx.to_csv('../data/cleaned/'+source+'export'+date+'.csv', index=False) 

print('done')
